# SORT tracking algorithm implementation


In [2]:
import numpy as np
from filterpy.kalman import KalmanFilter

In [3]:
def linear_assignment(cost_matrix):
    try:
        from scipy.optimize import linear_sum_assignment
    except ImportError:
        raise ImportError("Install scipy: pip install scipy")
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))

In [4]:
def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    intersection = w * h
    area1 = (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1])
    area2 = (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1])
    union = area1 + area2 - intersection
    return intersection / union

In [5]:
class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.eye(7)
        self.kf.H = np.zeros((4, 7))
        self.kf.H[:, :4] = np.eye(4)
        self.kf.P[4:, 4:] *= 1000.
        self.kf.P *= 10.

        self.kf.x[:4] = bbox.reshape(4, 1)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1

    def update(self, bbox):
        self.time_since_update = 0
        self.kf.update(bbox)

    def predict(self):
        self.kf.predict()
        self.time_since_update += 1
        return self.kf.x[:4].reshape(-1)

    def get_state(self):
        return self.kf.x[:4].reshape(-1)

In [6]:
class Sort:
    def __init__(self, max_age=1, min_hits=3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.trackers = []

    def update(self, dets):
        tracked_objects = []

        for t in self.trackers:
            t.predict()

        for det in dets:
            matched = False
            for t in self.trackers:
                if iou(det, t.get_state()) > 0.3:
                    t.update(det)
                    matched = True
                    break
            if not matched:
                self.trackers.append(KalmanBoxTracker(det))

        result = []
        for t in self.trackers:
            if t.time_since_update < self.max_age:
                result.append(np.concatenate((t.get_state(), [t.id])))

        return np.array(result)

# Object Detection and Tracking

In [7]:
import cv2
from ultralytics import YOLO
import numpy as np

In [8]:
# load YOLO model
model = YOLO("yolov8n.pt")

tracker = Sort()

100%|██████████| 6.23M/6.23M [00:05<00:00, 1.15MB/s]


In [13]:
# load video file
cap = cv2.VideoCapture("C:\\Users\\HP\\Object Detection and Tracking\\walking_people.mp4")  

In [15]:
while True:
    ret, frame = cap.read()
    if not ret:
        break  

    # YOLO detection
    results = model(frame, verbose=False)
    detections = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            if conf > 0.5:
                detections.append([x1, y1, x2, y2, conf])

    if len(detections) > 0:
        detections = np.array(detections)
    else:
        detections = np.empty((0, 5))

    # SORT tracking 
    tracked = tracker.update(detections[:, :4] if len(detections) else np.empty((0, 4)))

    # Draw tracking bounding boxes
    for d in tracked:
        x1, y1, x2, y2, obj_id = d.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {obj_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("YOLO + SORT Tracking", frame)

    # TO Quit the window use Q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()